In [41]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [169]:
import sklearn
sklearn.__version__

'1.0.2'

In [171]:
# !pip install -U scikit-learn --user

In [172]:
import sklearn
sklearn.__version__

'1.0.2'

In [42]:
datadir = "D:\DataScience-60\AttritionRatePredict"
filename = "Table_1.csv"
filePath  = os.path.join(datadir, filename)
df = pd.read_csv(filePath)

In [43]:
df.head()

,table id,name,phone number,Location,Emp. Group,Function,Gender,Tenure,Tenure Grp.,Experience (YY.MM),Marital Status,Age in YY.,Hiring Source,Promoted/Non Promoted,Job Role Match,Stay/Left
0,1,sid,9876544345,Pune,B2,Operation,Male,0.00,< =1,6.08,Single,27.12,Direct,Non Promoted,Yes,Left
1,2,sid,9876544345,Noida,B7,Support,Male,0.00,< =1,13.00,Marr.,38.08,Direct,Promoted,No,Stay
2,3,sid,9876544345,Bangalore,B3,Operation,Male,0.01,< =1,16.05,Marr.,36.04,Direct,Promoted,Yes,Stay
3,4,sid,9876544345,Noida,B2,Operation,Male,0.01,< =1,6.06,Marr.,32.07,Direct,Promoted,Yes,Stay
4,5,sid,9876544345,Lucknow,B2,Operation,Male,0.00,< =1,7.00,Marr.,32.05,Direct,Non Promoted,Yes,Stay


In [44]:
df.isna().sum()

table id                 0
name                     0
phone number             0
Location                 0
Emp. Group               0
Function                 0
Gender                   0
Tenure                   0
Tenure Grp.              0
Experience (YY.MM)       4
Marital Status           0
Age in YY.               0
Hiring Source            0
Promoted/Non Promoted    0
Job Role Match           2
Stay/Left                0
dtype: int64

In [54]:
df.dropna(axis=0, inplace=True)

In [55]:
df.isna().sum()

table id                 0
name                     0
phone number             0
Location                 0
Emp. Group               0
Function                 0
Gender                   0
Tenure                   0
Tenure Grp.              0
Experience (YY.MM)       0
Marital Status           0
Age in YY.               0
Hiring Source            0
Promoted/Non Promoted    0
Job Role Match           0
Stay/Left                0
dtype: int64

In [45]:
df['Job Role Match'].unique()

array(['Yes', 'No', nan], dtype=object)

In [46]:
df['Job Role Match'].value_counts()

Yes    482
No     418
Name: Job Role Match, dtype: int64

In [47]:
df.dropna()

,table id,name,phone number,Location,Emp. Group,Function,Gender,Tenure,Tenure Grp.,Experience (YY.MM),Marital Status,Age in YY.,Hiring Source,Promoted/Non Promoted,Job Role Match,Stay/Left
0,1,sid,9876544345,Pune,B2,Operation,Male,0.00,< =1,6.08,Single,27.12,Direct,Non Promoted,Yes,Left
1,2,sid,9876544345,Noida,B7,Support,Male,0.00,< =1,13.00,Marr.,38.08,Direct,Promoted,No,Stay
2,3,sid,9876544345,Bangalore,B3,Operation,Male,0.01,< =1,16.05,Marr.,36.04,Direct,Promoted,Yes,Stay
3,4,sid,9876544345,Noida,B2,Operation,Male,0.01,< =1,6.06,Marr.,32.07,Direct,Promoted,Yes,Stay
4,5,sid,9876544345,Lucknow,B2,Operation,Male,0.00,< =1,7.00,Marr.,32.05,Direct,Non Promoted,Yes,Stay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,897,Rahul,9876544345,Vijayawada,B1,Operation,Male,0.03,< =1,0.03,Single,24.06,Direct,Non Promoted,Yes,Stay
897,898,Rahul,9876544345,Vijayawada,B1,Operation,Female,0.03,< =1,0.03,Single,21.09,Direct,Non Promoted,Yes,Stay
898,899,Rahul,9876544345,Vijayawada,B1,Operation,Female,0.03,< =1,0.03,Single,22.02,Direct,Non Promoted,Yes,Left
900,901,Rahul,9876544345,Vijayawada,B1,Operation,Female,0.03,< =1,0.03,Single,22.02,Direct,Non Promoted,Yes,Stay


In [48]:
df.isnull().sum()

table id                 0
name                     0
phone number             0
Location                 0
Emp. Group               0
Function                 0
Gender                   0
Tenure                   0
Tenure Grp.              0
Experience (YY.MM)       4
Marital Status           0
Age in YY.               0
Hiring Source            0
Promoted/Non Promoted    0
Job Role Match           2
Stay/Left                0
dtype: int64

In [49]:
def mapping(x):
    x = None
    for i in df['Job Role Match']:
        if i != "Yes" and i != "No":
            x = 'Yes'
    return x

In [50]:
for i in df['Job Role Match']:
    if i != "Yes" and i != "No":
        print(i)
        print(type(i))
        print(type(np.nan))
    

nan
<class 'float'>
<class 'float'>
nan
<class 'float'>
<class 'float'>


In [51]:
df['Job Role Match'].value_counts()

Yes    482
No     418
Name: Job Role Match, dtype: int64

In [52]:
df['Job Role Match'].replace(np.nan,'Yes')

0      Yes
1       No
2      Yes
3      Yes
4      Yes
      ... 
897    Yes
898    Yes
899    Yes
900    Yes
901     No
Name: Job Role Match, Length: 902, dtype: object

In [53]:
df['Job Role Match'].unique()

array(['Yes', 'No', nan], dtype=object)

In [59]:
for col in df.columns:
    if df[col].dtype == object:
        print(col,":",df[col].unique())
        print(df[col].value_counts())
        print(">>>>>>>>>.-------.<<<<<<<< \n")

name : ['sid' 'shruti' 'Dwane' 'Arnab' 'Papu' 'Shiv' 'Rahul']
Arnab     309
Papu      201
Dwane     153
Shiv      127
shruti     74
sid        19
Rahul      13
Name: name, dtype: int64
>>>>>>>>>.-------.<<<<<<<< 

Location : ['Pune' 'Noida' 'Bangalore' 'Lucknow' 'Chennai' 'Hyderabad' 'Nagpur'
 'Madurai' 'Mumbai' 'Gurgaon' 'Kolkata' 'Vijayawada']
Chennai       255
Noida         236
Bangalore     210
Hyderabad      62
Pune           56
Madurai        29
Lucknow        20
Nagpur         14
Vijayawada      6
Mumbai          4
Gurgaon         3
Kolkata         1
Name: Location, dtype: int64
>>>>>>>>>.-------.<<<<<<<< 

Emp. Group : ['B2' 'B7' 'B3' 'B1' 'B5' 'B0' 'B4' 'B6' 'C3' 'D2']
B1    537
B2    276
B3     59
B0      8
B4      7
B5      4
B7      2
B6      1
C3      1
D2      1
Name: Emp. Group, dtype: int64
>>>>>>>>>.-------.<<<<<<<< 

Function : ['Operation' 'Support' 'Sales']
Operation    832
Support       52
Sales         12
Name: Function, dtype: int64
>>>>>>>>>.-------.<<<<<<<< 

G

In [65]:
location_dict_new = {
    'Chennai':       7,
    'Noida':         6,
    'Bangalore':     5,
    'Hyderabad':     4,
    'Pune':          3,
    'Madurai':       2,
    'Lucknow':       1,
    'other place':   0,
}

def location(x):
    if str(x) in location_dict_new.keys():
        return location_dict_new[str(x)]
    else:
        return location_dict_new['other place']
    
data_1 = df['Location'].apply(location)
df['New Location'] = data_1
df.head()

,table id,name,phone number,Location,Emp. Group,Function,Gender,Tenure,Tenure Grp.,Experience (YY.MM),Marital Status,Age in YY.,Hiring Source,Promoted/Non Promoted,Job Role Match,Stay/Left,New Location
0,1,sid,9876544345,Pune,B2,Operation,Male,0.00,< =1,6.08,Single,27.12,Direct,Non Promoted,Yes,Left,3
1,2,sid,9876544345,Noida,B7,Support,Male,0.00,< =1,13.00,Marr.,38.08,Direct,Promoted,No,Stay,6
2,3,sid,9876544345,Bangalore,B3,Operation,Male,0.01,< =1,16.05,Marr.,36.04,Direct,Promoted,Yes,Stay,5
3,4,sid,9876544345,Noida,B2,Operation,Male,0.01,< =1,6.06,Marr.,32.07,Direct,Promoted,Yes,Stay,6
4,5,sid,9876544345,Lucknow,B2,Operation,Male,0.00,< =1,7.00,Marr.,32.05,Direct,Non Promoted,Yes,Stay,1


In [66]:
df['New Location'].value_counts()

7    255
6    236
5    210
4     62
3     56
2     29
0     28
1     20
Name: New Location, dtype: int64

In [67]:
pd.get_dummies(df['Function'])

,Operation,Sales,Support
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
896,1,0,0
897,1,0,0
898,1,0,0
900,1,0,0


In [68]:
pd.get_dummies(df["Hiring Source"])

,Agency,Direct,Employee Referral
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
896,0,1,0
897,0,1,0
898,0,1,0
900,0,1,0


In [75]:
Maritial_dict = df['Marital Status'].value_counts()

In [76]:
def Mar(x):
    if str(x) in Maritial_dict  and Maritial_dict[str(x)] > 100:
        return x 
    else:
        return "other status"

In [69]:
df.columns

Index(['table id', 'name', 'phone number', 'Location', 'Emp. Group',
       'Function', 'Gender ', 'Tenure', 'Tenure Grp.', 'Experience (YY.MM)',
       'Marital Status', 'Age in YY.', 'Hiring Source',
       'Promoted/Non Promoted', 'Job Role Match', 'Stay/Left', 'New Location'],
      dtype='object')

In [73]:
df['Marital Status'].value_counts().keys()

Index(['Single', 'Marr.', 'Div.', 'NTBD', 'Sep.'], dtype='object')

In [74]:
df['Marital Status'].value_counts()

Single    534
Marr.     356
Div.        2
NTBD        2
Sep.        2
Name: Marital Status, dtype: int64

In [77]:
df['New Marital']= df['Marital Status'].apply(Mar)
df.head()

,table id,name,phone number,Location,Emp. Group,Function,Gender,Tenure,Tenure Grp.,Experience (YY.MM),Marital Status,Age in YY.,Hiring Source,Promoted/Non Promoted,Job Role Match,Stay/Left,New Location,New Marital
0,1,sid,9876544345,Pune,B2,Operation,Male,0.00,< =1,6.08,Single,27.12,Direct,Non Promoted,Yes,Left,3,Single
1,2,sid,9876544345,Noida,B7,Support,Male,0.00,< =1,13.00,Marr.,38.08,Direct,Promoted,No,Stay,6,Marr.
2,3,sid,9876544345,Bangalore,B3,Operation,Male,0.01,< =1,16.05,Marr.,36.04,Direct,Promoted,Yes,Stay,5,Marr.
3,4,sid,9876544345,Noida,B2,Operation,Male,0.01,< =1,6.06,Marr.,32.07,Direct,Promoted,Yes,Stay,6,Marr.
4,5,sid,9876544345,Lucknow,B2,Operation,Male,0.00,< =1,7.00,Marr.,32.05,Direct,Non Promoted,Yes,Stay,1,Marr.


In [78]:
df['New Marital'].value_counts()

Single          534
Marr.           356
other status      6
Name: New Marital, dtype: int64

In [83]:
def promo(x):
    if x == "Promoted":
        return int(1)
    else:
        return int(0)
df['New Promotion'] = df["Promoted/Non Promoted"].apply(promo)

In [84]:
df.head()

,table id,name,phone number,Location,Emp. Group,Function,Gender,Tenure,Tenure Grp.,Experience (YY.MM),Marital Status,Age in YY.,Hiring Source,Promoted/Non Promoted,Job Role Match,Stay/Left,New Location,New Marital,New Promotion
0,1,sid,9876544345,Pune,B2,Operation,Male,0.00,< =1,6.08,Single,27.12,Direct,Non Promoted,Yes,Left,3,Single,0
1,2,sid,9876544345,Noida,B7,Support,Male,0.00,< =1,13.00,Marr.,38.08,Direct,Promoted,No,Stay,6,Marr.,1
2,3,sid,9876544345,Bangalore,B3,Operation,Male,0.01,< =1,16.05,Marr.,36.04,Direct,Promoted,Yes,Stay,5,Marr.,1
3,4,sid,9876544345,Noida,B2,Operation,Male,0.01,< =1,6.06,Marr.,32.07,Direct,Promoted,Yes,Stay,6,Marr.,1
4,5,sid,9876544345,Lucknow,B2,Operation,Male,0.00,< =1,7.00,Marr.,32.05,Direct,Non Promoted,Yes,Stay,1,Marr.,0


In [86]:
df['Emp. Group'].value_counts()

B1    537
B2    276
B3     59
B0      8
B4      7
B5      4
B7      2
B6      1
C3      1
D2      1
Name: Emp. Group, dtype: int64

In [87]:
emp_dict_new = {
    'B1': 4,
    'B2': 3,
    'B3': 2,
    'other group': 1,    
}


def emp(x):
    if str(x) in emp_dict_new.keys():
        return str(x)
    else:
        return "other group"
    
df['New EMP']= df["Emp. Group"].apply(emp)
df.head()

,table id,name,phone number,Location,Emp. Group,Function,Gender,Tenure,Tenure Grp.,Experience (YY.MM),Marital Status,Age in YY.,Hiring Source,Promoted/Non Promoted,Job Role Match,Stay/Left,New Location,New Marital,New Promotion,New EMP
0,1,sid,9876544345,Pune,B2,Operation,Male,0.00,< =1,6.08,Single,27.12,Direct,Non Promoted,Yes,Left,3,Single,0,B2
1,2,sid,9876544345,Noida,B7,Support,Male,0.00,< =1,13.00,Marr.,38.08,Direct,Promoted,No,Stay,6,Marr.,1,other group
2,3,sid,9876544345,Bangalore,B3,Operation,Male,0.01,< =1,16.05,Marr.,36.04,Direct,Promoted,Yes,Stay,5,Marr.,1,B3
3,4,sid,9876544345,Noida,B2,Operation,Male,0.01,< =1,6.06,Marr.,32.07,Direct,Promoted,Yes,Stay,6,Marr.,1,B2
4,5,sid,9876544345,Lucknow,B2,Operation,Male,0.00,< =1,7.00,Marr.,32.05,Direct,Non Promoted,Yes,Stay,1,Marr.,0,B2


In [88]:
def job(x):
    if x == "Yes":
        return int(1)
    else:
        return int(0)
    
df['New Job Role Match'] = df['Job Role Match'].apply(job)
df.head()

,table id,name,phone number,Location,Emp. Group,Function,Gender,Tenure,Tenure Grp.,Experience (YY.MM),...,Age in YY.,Hiring Source,Promoted/Non Promoted,Job Role Match,Stay/Left,New Location,New Marital,New Promotion,New EMP,New Job Role Match
0,1,sid,9876544345,Pune,B2,Operation,Male,0.00,< =1,6.08,...,27.12,Direct,Non Promoted,Yes,Left,3,Single,0,B2,1
1,2,sid,9876544345,Noida,B7,Support,Male,0.00,< =1,13.00,...,38.08,Direct,Promoted,No,Stay,6,Marr.,1,other group,0
2,3,sid,9876544345,Bangalore,B3,Operation,Male,0.01,< =1,16.05,...,36.04,Direct,Promoted,Yes,Stay,5,Marr.,1,B3,1
3,4,sid,9876544345,Noida,B2,Operation,Male,0.01,< =1,6.06,...,32.07,Direct,Promoted,Yes,Stay,6,Marr.,1,B2,1
4,5,sid,9876544345,Lucknow,B2,Operation,Male,0.00,< =1,7.00,...,32.05,Direct,Non Promoted,Yes,Stay,1,Marr.,0,B2,1


In [90]:
df['Gender '].value_counts()

Male      656
Female    234
other       6
Name: Gender , dtype: int64

In [91]:
df.columns

Index(['table id', 'name', 'phone number', 'Location', 'Emp. Group',
       'Function', 'Gender ', 'Tenure', 'Tenure Grp.', 'Experience (YY.MM)',
       'Marital Status', 'Age in YY.', 'Hiring Source',
       'Promoted/Non Promoted', 'Job Role Match', 'Stay/Left', 'New Location',
       'New Marital', 'New Promotion', 'New EMP', 'New Job Role Match'],
      dtype='object')

In [93]:
for col in df.columns:
    if df[col].dtype == object:
        print(col)

name
Location
Emp. Group
Function
Gender 
Tenure Grp.
Marital Status
Hiring Source
Promoted/Non Promoted
Job Role Match
Stay/Left
New Marital
New EMP


In [95]:
# name_dummy = pd.get_dummies(df['name'])
tenure_grp_dummy = pd.get_dummies(df['Tenure Grp.'])
gender_dummy = pd.get_dummies(df['Gender '])
marit_dummy = pd.get_dummies(df['New Marital'])
func_dummy = pd.get_dummies(df['Function'])
hiring_source_dummy = pd.get_dummies(df['Hiring Source'])
emp_dummy = pd.get_dummies(df['New EMP'])

In [96]:
df1 = pd.concat([df, tenure_grp_dummy,gender_dummy, marit_dummy,
                func_dummy, hiring_source_dummy,
                emp_dummy,], axis=1)

In [97]:
df1.shape

(896, 39)

In [99]:
for col in df1:
    if df1[col].dtype != object:
        print( col )

table id
phone number
Tenure
Experience (YY.MM)
Age in YY.
New Location
New Promotion
New Job Role Match
< =1
> 1 & < =3
Female
Male
other
Marr.
Single
other status
Operation
Sales
Support
Agency
Direct
Employee Referral
B1
B2
B3
other group


In [101]:
df1.drop(["table id", "name", "Marital Status","Promoted/Non Promoted","Function","Emp. Group","Job Role Match","Location"
              ,"Hiring Source","Gender ", 'Tenure','New Marital', 'New EMP'],axis=1,inplace=True)

In [102]:
df1.shape

(896, 26)

In [107]:
df1['Stay/Left'].value_counts()

Stay    592
Left    304
Name: Stay/Left, dtype: int64

In [108]:
def target(x):
    if x == "Stay":
        return int(1)
    else:
        return int(0)

In [109]:
df1['target'] = df1['Stay/Left'].apply(target)

In [131]:
df1.drop(['Tenure Grp.'], axis=1, inplace=True)

In [112]:
df1.drop(['Stay/Left'], axis=1, inplace=True)

In [132]:
df1.to_csv("processed_table.csv")

In [133]:
df2 = pd.read_csv('processed_table.csv')

In [134]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          896 non-null    int64  
 1   phone number        896 non-null    int64  
 2   Experience (YY.MM)  896 non-null    float64
 3   Age in YY.          896 non-null    float64
 4   New Location        896 non-null    int64  
 5   New Promotion       896 non-null    int64  
 6   New Job Role Match  896 non-null    int64  
 7   < =1                896 non-null    int64  
 8   > 1 & < =3          896 non-null    int64  
 9   Female              896 non-null    int64  
 10  Male                896 non-null    int64  
 11  other               896 non-null    int64  
 12  Marr.               896 non-null    int64  
 13  Single              896 non-null    int64  
 14  other status        896 non-null    int64  
 15  Operation           896 non-null    int64  
 16  Sales   

In [135]:
X = df2.drop('target', axis=1)
y = df2['target']

In [117]:
from sklearn.model_selection import train_test_split

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=4)

In [137]:
X_train.shape

(716, 25)

In [138]:
y_train.shape

(716,)

In [142]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score

In [125]:
logReg = LogisticRegression(C=0.1, solver='liblinear')
DecTree = DecisionTreeClassifier()
RF = RandomForestClassifier()
GausNB = GaussianNB()

In [143]:
for model, name in zip([logReg, DecTree, RF, GausNB],
               ['LogisticRegression', 'DecisionTree','RandomForest','GaussianNB']):
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_accu = accuracy_score(y_train,train_pred)
    test_accu = accuracy_score(y_test, test_pred)
    train_msg = "[%s] training accuracy: %f" % (name, train_accu)
    test_msg = "[%s] testing accuracy: %f" % (name, test_accu)
    print(train_msg)
    print(test_msg)

[LogisticRegression] training accuracy: 0.662011
[LogisticRegression] testing accuracy: 0.655556
[DecisionTree] training accuracy: 1.000000
[DecisionTree] testing accuracy: 0.855556
[RandomForest] training accuracy: 1.000000
[RandomForest] testing accuracy: 0.883333
[GaussianNB] training accuracy: 0.877095
[GaussianNB] testing accuracy: 0.811111


In [148]:
model = logReg.fit(X_train, y_train)
y_pred = model.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(score)

0.6555555555555556


In [150]:
import pickle

model = logReg.fit(X_train, y_train)
file = 'evaluated_model.pkl'
pickle.dump(model, open(file, "wb"))

In [155]:
with open('evaluated_model.pkl', 'rb') as file:
    evail_model = pickle.load(file)

In [154]:
test_s = X_train.iloc[10:11,:].values

In [166]:
for i in range(len(X_train)):
    if evail_model.predict(X_train.iloc[i:i+1,:])[0] == 0:
        print("hello")

In [158]:
evail_model.predict(test_s)

C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([1], dtype=int64)

In [162]:
y_train

385    1
215    0
399    1
411    0
354    1
      ..
360    0
709    0
439    1
174    0
122    1
Name: target, Length: 716, dtype: int64

In [168]:
import sklearn
sklearn.__version__

'1.0.2'